# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

    # Read csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # Create a CSV reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # Extract each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Uncomment the code below to print the total number of rows 
print(len(full_data_rows_list))

# Uncomment the code below to print the contents of event data rows
#print(full_data_rows_list)

# Creating a smaller event data CSV file called event_datafile_full csv that will be used to insert data into the
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra code 

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance on your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try: 
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkifydb
        WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Create tables to run the following queries

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



#### Create table for query 1

In [8]:
query = "CREATE TABLE IF NOT EXISTS session_songs"
query = query + "(sessionId int, \
                  itemInSession int, \
                  artist text, \
                  songTitle text, \
                  songLength float, \
                  PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)          
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_songs (sessionId, itemInSession, artist, songTitle, songLength)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Create table for query 2

In [9]:
query = "CREATE TABLE IF NOT EXISTS user_sessions"
query = query + "(userId int, \
                  sessionId int, \
                  itemInSession int, \
                  artist text, \
                  songTitle text, \
                  firstName text, \
                  lastName text, \
                  PRIMARY KEY ((userId, sessionId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)      
                 
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_sessions (userId, sessionId, itemInSession, artist, songTitle, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Create table for query 3

In [10]:
query = "CREATE TABLE IF NOT EXISTS song_listeners"
query = query + "(songTitle text, \
                  userId int, \
                  firstName text, \
                  lastName text, \
                  PRIMARY KEY (songTitle, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)      
                 
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listeners (songTitle, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Run the queries to verify that the data have been inserted into each table

In [11]:
# Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
query = "SELECT artist, songtitle, songlength from session_songs WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.songtitle, row.songlength)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [12]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = "SELECT artist, songtitle, firstname, lastname from user_sessions WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.songtitle, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [13]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "SELECT firstname, lastname from song_listeners WHERE songTitle = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [14]:
try:
    session.execute("DROP TABLE IF EXISTS session_songs ")
except Execption as e:
    print(e)
    
try:
    session.execute("DROP TABLE IF EXISTS user_sessions ")
except Execption as e:
    print(e)
    
try:
    session.execute("DROP TABLE IF EXISTS song_listeners")
except Execption as e:
    print(e) 

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()